<a href="https://colab.research.google.com/github/Aarthi1508/yk-aws-application/blob/main/aws_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import the required packages**

In [1]:
import os
import rasterio  # Import the rasterio module for reading and writing geospatial raster datasets
import numpy as np  # Import the NumPy module for numerical computing
import os  # Import the os module for interacting with the operating system
import matplotlib.pyplot as plt  # Import the matplotlib.pyplot module for creating static plots and visualizations
import pandas as pd  # Import the pandas module for data manipulation and analysis

C:\Users\guard\AppData\Local\Temp\ipykernel_12192\3327126247.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd  # Import the pandas module for data manipulation and analysis


### Load customized elevation map

1. Downlaoad Digital Eleveation Model (DEM) you interest 

        - One valid source is USGS-EarthExpolorer (https://earthexplorer.usgs.gov/). Here is the tutorial link (https://www.youtube.com/watch?v=NQg0g9ObhXE)

2. Extract subarea of interest from the doloaded DSM (e.g., 400x400)

3. [Aarthi] ~~~ Shadow Simulation + web update if possible

4. Image to GIF

### 1. Load the downloaded DSM

In [2]:
def read_geotiff(input_file):
    # Open the GeoTIFF file
    with rasterio.open(input_file, 'r') as src:
        # Read metadata
        width = src.width
        height = src.height
        count = src.count  # Number of bands
        crs = src.crs.to_epsg()  # EPSG code of the coordinate reference system
        transform = src.transform  # Affine transformation parameters

        # Read each band
        data_arrays = [src.read(band_index + 1) for band_index in range(count)]
        data_names = [src.descriptions[band_index] for band_index in range(count)]

    return {
        'width': width,
        'height': height,
        'count': count,
        'crs': crs,
        'transform': transform,
        'data_arrays': data_arrays,
        'data_names': data_names
    }

In [3]:
# Path to GeoTIFF file
input_tif_file_path = './webapp/data/n29_w096_1arc_v3.tif' # Replace with the path to your GeoTIFF file

# Read GeoTIFF file
geotiff_data = read_geotiff(os.path.join(input_tif_file_path))

# Access data from geotiff_data dictionary
width = geotiff_data['width']
height = geotiff_data['height']
count = geotiff_data['count']
crs = geotiff_data['crs']
transform = geotiff_data['transform']
data_arrays = geotiff_data['data_arrays']
data_names = geotiff_data['data_names']

# Print metadata information
print(f"Width: {width}, Height: {height}, Bands: {count}")
print(f"Coordinate Reference System (CRS): EPSG:{crs}")
print(f"Affine Transformation: {transform}")

print()

# Accessing individual bands
for band_index, (data_array, data_name) in enumerate(zip(data_arrays, data_names)):
    print(f"Band {band_index + 1} - ({data_name}):")
    print(data_array.shape)

dsm_raw = data_arrays[0]
print(type(dsm_raw))
print("DSM shape", dsm_raw.shape)
dsm_raw

Width: 3601, Height: 3601, Bands: 1
Coordinate Reference System (CRS): EPSG:4326
Affine Transformation: | 0.00, 0.00,-96.00|
| 0.00,-0.00, 30.00|
| 0.00, 0.00, 1.00|

Band 1 - (None):
(3601, 3601)
<class 'numpy.ndarray'>
DSM shape (3601, 3601)


array([[59, 59, 59, ..., 22, 22, 22],
       [61, 60, 59, ..., 22, 22, 22],
       [64, 63, 61, ..., 21, 21, 21],
       ...,
       [21, 21, 21, ...,  0,  0,  0],
       [22, 23, 23, ...,  0,  0,  0],
       [22, 23, 24, ...,  0,  0,  0]], dtype=int16)

### 2. Extract subarea of interest from the doloaded DSM (e.g., 400x400)

In [4]:
dsm = dsm_raw[:400, :400]

print(type(dsm))
print("DSM shape", dsm.shape)
dsm

<class 'numpy.ndarray'>
DSM shape (400, 400)


array([[59, 59, 59, ..., 72, 72, 72],
       [61, 60, 59, ..., 73, 72, 73],
       [64, 63, 61, ..., 74, 73, 73],
       ...,
       [50, 49, 49, ..., 56, 56, 56],
       [49, 48, 49, ..., 56, 56, 54],
       [49, 50, 50, ..., 56, 55, 53]], dtype=int16)

### 3. [Aarthi] ~~~ Shadow Simulation + web update if possible

### 4. Image to Tif file

In [5]:
from img2gif import *

In [6]:
# folder path
img_dir_path = './img/' # Dir where img files of the shadow are stored
result_dir_path = './result/' # Dir where gif of the shadow will be stored

if not os.path.exists(img_dir_path):
    os.mkdir(img_dir_path)

if not os.path.exists(result_dir_path):
    os.mkdir(result_dir_path)

# list to store files
res = []

# Iterate directory
for path in os.listdir(img_dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(img_dir_path, path)):
        res.append(path)

res.sort()
print(res)

['hour16minute00.png', 'hour16minute30.png', 'hour17minute00.png', 'hour17minute30.png', 'hour18minute00.png']


In [7]:
import PIL.Image

ims = []
for i in range(len(res)):
    frame = PIL.Image.open(os.path.join(img_dir_path, res[i]))
    ims.append(frame)

In [8]:
save_transparent_gif(images = ims,durations=300, save_file = './Result/Shadow.gif')

In [9]:
# Check the saved file at './Result/Shadow.gif'